In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, GridBox, Layout, VBox
import plotly.graph_objs as go

%matplotlib widget

In [ ]:
fig = go.FigureWidget()

"""
Submits context and creates pie chart
Updates sliders for pie chart accordingly
"""
def submit(forest, urban, pasture, suburb):
    data = [forest, urban, pasture, suburb]
    fig.data = []
    fig.add_pie(values=data, labels = ["Forest", "Urban", "Pasture", "Suburb"])
    dforest.value = forest
    durban.value = urban
    dpasture.value = pasture
    dsuburb.value = suburb

"""
Real-time updater for pie chart
"""
def update(dforest, durban, dpasture, dsuburb):
    with fig.batch_update():
        if len(fig["data"]) > 0:
            fig["data"][0]["values"] = [dforest, durban, dpasture, dsuburb]


# Construct widgets
# TODO: Handle unusuable land
forest=widgets.FloatText(value=0.0,description='Forest',disabled=False)
urban=widgets.FloatText(value=0.0,description='Urban',disabled=False)
pasture=widgets.FloatText(value=0.0,description='Pasture',disabled=False)
suburb=widgets.FloatText(value=0.0,description='Suburb',disabled=False)

dforest=widgets.FloatSlider(value=0.0, description='Forest Diff')
durban=widgets.FloatSlider(value=0.0,description='Urban',disabled=False)
dpasture=widgets.FloatSlider(value=0.0,description='Pasture',disabled=False)
dsuburb=widgets.FloatSlider(value=0.0,description='Suburb',disabled=False)


# Display interactables
# TODO: add titles, make layout prettier
context_interact = interact_manual(submit, forest=forest, urban=urban, pasture=pasture, suburb=suburb)
context_interact.widget.children[-2].description = "Prescribe"

# TODO: have them sum to 1 and make lock ticks
update_interact = interact(update, dforest=dforest, durban=durban, dpasture=dpasture, dsuburb=dsuburb)

# Display fig
#TODO: how do I clean this up?
display(fig)